In [1]:
#Import required Libraries
import pandas

import sklearn
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import MSELoss

from HousePriceDataset import HousePriceDataset
from PredictorModel import HousePricePredictor
from ModelTrainingEpoch import model_training_epoch
from ModelValidationEpoch import model_validation_epoch

In [2]:
# load the dataset
housing_price_dataset = pandas.read_csv("housing.csv")
housing_price_dataset.columns = housing_price_dataset.columns.str.strip()

housing_data_input = housing_price_dataset.drop(columns=["median_house_value", "ocean_proximity"])
housing_data_output = housing_price_dataset["median_house_value"]

train_df, test_df = train_test_split(
    housing_price_dataset,
    test_size=0.25,
    random_state=42,
    shuffle=True,
)

X_train = train_df.drop(columns=["median_house_value", "ocean_proximity"], errors='ignore')
Y_train = train_df["median_house_value"]

X_test = test_df.drop(columns=["median_house_value", "ocean_proximity"], errors='ignore')
Y_test = test_df["median_house_value"]

#X_train

In [3]:
train_dataset = HousePriceDataset(X_train.values, Y_train.values)
val_dataset = HousePriceDataset(X_test.values, Y_test.values)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = HousePricePredictor(input_dim=X_train.shape[1])
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)


In [5]:
num_epochs = 50

for epoch in range(num_epochs):
    train_loss = model_training_epoch(
        model,
        train_loader,
        loss_fn,
        optimizer,
        device
    )

    val_loss = model_validation_epoch(
        model,
        val_loader,
        loss_fn,
        device
    )

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"Train Loss: {train_loss:.4f} "
        f"Val Loss: {val_loss:.4f}"
    )


Epoch [1/50] Train Loss: 56209520068.7603 Val Loss: nan
Epoch [2/50] Train Loss: 56207128774.8760 Val Loss: nan
Epoch [3/50] Train Loss: 56196467356.5620 Val Loss: nan
Epoch [4/50] Train Loss: 56186054829.4876 Val Loss: nan
Epoch [5/50] Train Loss: 56187666901.6860 Val Loss: nan
Epoch [6/50] Train Loss: 56173400977.9835 Val Loss: nan
Epoch [7/50] Train Loss: 56183410726.0826 Val Loss: nan
Epoch [8/50] Train Loss: 56180030172.0331 Val Loss: nan
Epoch [9/50] Train Loss: 56165552618.8430 Val Loss: nan
Epoch [10/50] Train Loss: 56154355563.9008 Val Loss: nan
Epoch [11/50] Train Loss: 56145321670.8760 Val Loss: nan
Epoch [12/50] Train Loss: 56159480565.4215 Val Loss: nan
Epoch [13/50] Train Loss: 56157174640.1322 Val Loss: nan
Epoch [14/50] Train Loss: 56145286732.1653 Val Loss: nan
Epoch [15/50] Train Loss: 56153151107.1736 Val Loss: nan
Epoch [16/50] Train Loss: 56138115114.3140 Val Loss: nan
Epoch [17/50] Train Loss: 56127094047.7355 Val Loss: nan
Epoch [18/50] Train Loss: 56137610041.12